In [5]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

import os
import re
import json
import string
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

max_length = 250

In [6]:
from transformers import BertTokenizer
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# config = BertConfig.from_pretrained("bert-base-uncased", num_labels=3, do_lower_case=True)
# bert_model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)


In [16]:
def convert_example_to_feature(text):  
  return tokenizer.encode_plus(text, 
                truncation=True,
#                 add_special_tokens = True,
                max_length = max_length, 
                padding= 'max_length', # 
                return_attention_mask = True,
              )

In [25]:
def map_example_to_dict(input_ids, attention_masks, label):
  return {
      "input_ids": input_ids,
#       "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(ds, limit=-1):

  input_ids_list = []
#   token_type_ids_list = []
  attention_mask_list = []
  label_list = []
    
  for index, row in df.iterrows():

    bert_input = convert_example_to_feature(row['Text'])
    input_ids_list.append(bert_input['input_ids'])
#     token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    label_list.append([row['Category']])

  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, label_list)).map(map_example_to_dict)


In [32]:
df = pd.read_csv('data.csv')
df.head()

,Text,Category
0,"As someone who has worked with many museums, I...",0
1,I am actually horrified this place is still in...,0
2,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",0
3,"If I could give less than one star, that would...",0
4,10pm on a super bowl Sunday and they're alread...,0


In [33]:
from sklearn.utils import shuffle
df = shuffle(df)

In [34]:
df.dropna()

,Text,Category
143094,Clam Chowder - delish - and considering we are...,1
146596,The rueben is awesome. \n\nThe bison burger l...,1
5780,ALWAYS trying to upsell me on a number of serv...,0
34187,Slum... Run don't walk away from this place. T...,0
71833,I had a great first dining experience at Anejo...,2
...,...,...
92902,"Awesome everything!! Service, food, friendlin...",2
125473,My gf and I came here on Tuesday night for the...,1
93308,Brunch review. So excited there is a good plac...,2
17191,Worst service I've ever had in Calgary. The p...,0


In [35]:
df = df[:50000]

In [36]:
df.shape

(50000, 2)

In [37]:
df['Category'].value_counts()

0    16731
2    16665
1    16604
Name: Category, dtype: int64

In [ ]:
df = None

In [38]:
ds_train = encode_examples(df).batch(16)

In [30]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf
learning_rate = 2e-5
number_of_epochs = 1


model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use i

In [31]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  2307      
_________________________________________________________________
dropout_39 (Dropout)         multiple                  0         
Total params: 66,955,779
Trainable params: 66,955,779
Non-trainable params: 0
_________________________________________________________________


In [39]:
bert_history = model.fit(ds_train, epochs=1)

3125/3125 [==============================] - 1146s 367ms/step - loss: 0.5129 - sparse_categorical_accuracy: 0.7795


In [58]:
sample = convert_example_to_feature("this is horrible, i don't like it")
ds = tf.data.Dataset.from_tensor_slices(([sample['input_ids']], [sample['attention_mask']], [0])).map(map_example_to_dict)
pred = model.predict(ds.batch(1))
print(np.argmax(pred[0][0]))

0
